In [6]:
import sys
sys.path.append('../python_scripts/')
import os
import datasets, train
import tensorflow as tf

# Complete notebook to present data for IDVOR
Compiled and fixed by Jason, base code by Sam

## Baseline Variability
We should quantify baseline variability before how measuring how much variability comes from randomization seed and dataset shuffling

First, we'll train an instance of All-CNN-C on CIFAR10. 

In [17]:
# Make GPU training deterministic
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['PYTHONHASHSEED'] = str(0)

# Set seeds
shuffleSeed = 2020
weightSeed = 2021

trainData, testData = datasets.make_train_data(shuffle_seed=shuffleSeed, augment=True)
x_predict, y_predict = datasets.make_predict_data(testData)

Making train data...
GCN...
ZCA...
Done!
Making test data...
Done!


We preprocess the images the same way that the original authors do, global contrast normalization and whitened with ZCA. During training, we augment the data the same way: horizontal flipping and random translation (5 pixels in any direction). 

In [18]:
modelPath = 'masterOutput/baseline/w'+str(weightSeed)+'s'+str(shuffleSeed)+'.pb'

if os.path.exists(modelPath): # The model already exists, load it.
    print('Loading model')
else: # Train new model
    print('Trianing new model')
    model = train.init_all_cnn_c(seed=weightSeed)

    # Set flag to true if converges to local min
    abort = False
    history = model.fit(
        x=trainData,
        epochs=350,
        validation_data=testData.prefetch(tf.data.experimental.AUTOTUNE)\
                     .batch(128),
        callbacks=[train.LR_Callback, train.Early_Abort_Callback()])

    if not abort:
        tensorflow.keras.models.save_model(model, modelPath)
    else:
        raise ValueException('Model hit local minimum!')

Trianing new model
Epoch 1/350
117/391 [=======>......................] - ETA: 5:02 - loss: 2.3293 - accuracy: 0.0944

KeyboardInterrupt: 

'2.3.1'